In [1]:
# Dependencies and config file
import csv
import matplotlib.pyplot as plt
import requests
import pandas as pd
from config import api_key
import openweathermapy as owm
from citipy import citipy
from random import randint
import numpy as np

In [2]:
latitude_list = []
for x in range(500):
    # Randomly select 500 unique latitudes
    latitude_list.append(np.random.uniform(low=-180.0, high=180, size=None))
    #print(latitude_list)
    # this works but it is a very long list 
latitude_list_series = pd.Series(latitude_list)
# Then add the values to the DataFrame:
#city_df['new_col'] = latitude_list_series.values

In [3]:
longitude_list = []
for x in range(500):
    # Randomly select 500 unique longitudes
    longitude_list.append(np.random.uniform(low=-90, high = 90))
    #print(longitude)

# Convert list to series
longitude_list_series = pd.Series(longitude_list)

In [4]:
lat_and_long = pd.concat([latitude_list_series, longitude_list_series], axis=1)
#lat_and_long.head(10)
lat_and_long = lat_and_long.set_index(0)
lat_and_long.head(10)

,1
0,
17.935375,38.533951
-158.364384,-37.356530
129.561150,-23.936578
125.399645,60.106245
-162.970411,6.728342
-98.818828,-19.241323
88.221885,-42.272819
-99.627588,45.769641
8.597225,71.375103


In [19]:
# Blank column for holiding the city name
cities = []

#row_counter = 0
for index, row in lat_and_long.iterrows():
    city = citipy.nearest_city(index, row[1])
    city_list = city.city_name
    cities.append(city_list)
#print(len(city_list))
print(cities)

['tawkar', 'ushuaia', 'illoqqortoormiut', 'belushya guba', 'bredasdorp', 'ushuaia', 'upernavik', 'port alfred', 'kavaratti', 'ushuaia', 'longyearbyen', 'luanda', 'saint george', 'ushuaia', 'ushuaia', 'moa', 'ushuaia', 'quelimane', 'port alfred', 'port alfred', 'ushuaia', 'qaanaaq', 'mahebourg', 'illoqqortoormiut', 'georgetown', 'jamestown', 'ushuaia', 'thompson', 'joshimath', 'bredasdorp', 'barentsburg', 'illoqqortoormiut', 'kaabong', 'qaanaaq', 'albany', 'ushuaia', 'ushuaia', 'aswan', 'qaanaaq', 'mudyuga', 'ushuaia', 'sioux lookout', 'ushuaia', 'narsaq', 'ushuaia', 'hermanus', 'narsaq', 'ushuaia', 'hermanus', 'dikson', 'vostochnyy', 'ushuaia', 'mar del plata', 'dikson', 'port alfred', 'attawapiskat', 'ushuaia', 'nizhnedevitsk', 'arraial do cabo', 'vryburg', 'longyearbyen', 'dikson', 'longyearbyen', 'barentsburg', 'port elizabeth', 'abonnema', 'mahebourg', 'ushuaia', 'busselton', 'longyearbyen', 'ushuaia', 'port elizabeth', 'tumannyy', 'ushuaia', 'busselton', 'qaanaaq', 'clyde river', 

In [6]:
# empty dataframe is this needed?
#d = {'city': [""], 'latitude': [""], 'longitude': [""]}
#city_df = pd.DataFrame(data=d, index=[0,1,2,3,4,5,6,500])
#city_df.head(25)
#city_data = pd.DataFrame(data=None, index=None, columns=None, dtype=None, copy=False)

In [7]:
# Create settings dictionary 
#settings = {"units": "imperial", "APPID": api_key}

In [8]:
# Randomly select **at least** 500 unique (non-repeat) cities based on latitude and longitude.
# Perform a weather check on each of the cities using a series of successive API calls. 
# Include a print log of each city as it's being processed with the city number, city name, and requested URL.
# Save both a CSV of all data retrieved and png images for each scatter plot.

In [9]:
#Your objective is to build a series of scatter plots to showcase the following relationships:

#* Temperature (F) vs. Latitude
#* Humidity (%) vs. Latitude
#* Cloudiness (%) vs. Latitude
#* Wind Speed (mph) vs. Latitude

In [10]:
# Build a scatter plot for each data type
# Example from other activity
#plt.scatter(census_data_complete["Bank Count"].astype(int),
            #census_data_complete["Unemployment Rate"],
            #edgecolor="black", linewidths=1, marker="o",
            #alpha=0.8, label="Zip Codes")

# Label the graph and set axis limits
#plt.title("Unemployment Rate vs. Bank Count by Zip Code")
#plt.ylabel("Unemployment Rate (%)")
#plt.xlabel("Bank Count")
#plt.grid(True)
#plt.xlim([-2.5, 202])
#plt.ylim([-2.5, 40])

# Save the figure
#plt.savefig("UnemploymentRate_BankCount.png")

# Show plot
#plt.show()